In [ ]:
import sys
sys.path.append('..')

In [ ]:
from helper_fun import *
import helper_fun
from Split_functions_classify.splits_two import *
from Split_functions_classify.splits_three import *
from Split_functions_classify.splits_ten import *

In [ ]:
compounds = pd.read_csv('../COVID_MOONSHOT/compounds_filtered.csv')
compounds.head(2)

### Adding columns for our different amounts of bins we will use to classify ###

In [ ]:
compounds = two_split(compounds)
compounds = three_split(compounds)
compounds = ten_split(compounds)

Quick check

In [ ]:
compounds.head(1)

Now we set our SEED, for reproducible results

In [ ]:
SEED = 22
seed_everything(SEED)

Add in our compound fingerprints

In [ ]:
compound_df = compounds.copy()

In [ ]:
compound_df["maccs"] = compound_df["SMILES"].apply(smiles_to_fp,)

In [ ]:
compound_df

In [ ]:
from sklearn import svm, metrics, clone
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import auc, accuracy_score, recall_score
from sklearn.metrics import roc_curve, roc_auc_score

Preprocessing the data using a random split

Two Bin Split

In [ ]:
fingerprint_to_model = compound_df.maccs.to_list()
label_to_model_2 = compound_df.bin_2.to_list()

# Split data randomly in train and test set
# note that we use test/train_x for the respective fingerprint splits
# and test/train_y for the respective label splits
(
    static_train_x_2,
    static_test_x_2,
    static_train_y_2,
    static_test_y_2,
) = train_test_split(fingerprint_to_model, label_to_model_2, test_size=0.2, random_state=SEED)
splits = [static_train_x_2, static_test_x_2, static_train_y_2, static_test_y_2]
# NBVAL_CHECK_OUTPUT
print("Training data size:", len(static_train_x_2))
print("Test data size:", len(static_test_x_2))

3 Bin Split

In [ ]:
label_to_model_3 = compound_df.bin_3.to_list()

# Split data randomly in train and test set
# note that we use test/train_x for the respective fingerprint splits
# and test/train_y for the respective label splits
(
    static_train_x_3,
    static_test_x_3,
    static_train_y_3,
    static_test_y_3,
) = train_test_split(fingerprint_to_model, label_to_model_3, test_size=0.2, random_state=SEED)
splits = [static_train_x_3, static_test_x_3, static_train_y_3, static_test_y_3]
# NBVAL_CHECK_OUTPUT
print("Training data size:", len(static_train_x_3))
print("Test data size:", len(static_test_x_3))

10 Bin Split

In [ ]:
label_to_model_10 = compound_df.bin_10.to_list()

# Split data randomly in train and test set
# note that we use test/train_x for the respective fingerprint splits
# and test/train_y for the respective label splits
(
    static_train_x_10,
    static_test_x_10,
    static_train_y_10,
    static_test_y_10,
) = train_test_split(fingerprint_to_model, label_to_model_10, test_size=0.2, random_state=SEED)
splits = [static_train_x_10, static_test_x_10, static_train_y_10, static_test_y_10]
# NBVAL_CHECK_OUTPUT
print("Training data size:", len(static_train_x_10))
print("Test data size:", len(static_test_x_10))

## Neural Network Classifier ##

In [ ]:
# Specify model
model_ANN = MLPClassifier(hidden_layer_sizes=(5, 3), random_state=SEED)